In [2]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *



training = pd.read_csv(r"IMDBPROCESSED.csv", encoding="utf8")


#remove nonASCIICharacters
training["text"] = training.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)
training



,text,sentiment,text_tb_pol,text_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos,text_vader_comp
0,One of the other reviewers has mentioned that ...,1,0.023433,0.490369,0.179,0.756,0.064,-0.9916
1,A wonderful little production. <br /><br />The...,1,0.109722,0.559343,0.052,0.773,0.176,0.9670
2,I thought this was a wonderful way to spend ti...,1,0.354008,0.658730,0.114,0.688,0.198,0.9519
3,Basically there's a family where a little boy ...,0,-0.057813,0.454167,0.125,0.816,0.059,-0.9213
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,0.217952,0.452916,0.050,0.806,0.144,0.9744
...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,1,0.394425,0.522897,0.045,0.765,0.189,0.9886
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,-0.276190,0.642857,0.160,0.730,0.110,-0.6837
49997,I am a Catholic taught in parochial elementary...,0,0.056984,0.395538,0.181,0.704,0.115,-0.9734
49998,I'm going to have to disagree with the previou...,0,-0.048663,0.447034,0.116,0.804,0.080,-0.8657


In [3]:
training

,text,sentiment,text_tb_pol,text_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos,text_vader_comp
0,One of the other reviewers has mentioned that ...,1,0.023433,0.490369,0.179,0.756,0.064,-0.9916
1,A wonderful little production. <br /><br />The...,1,0.109722,0.559343,0.052,0.773,0.176,0.9670
2,I thought this was a wonderful way to spend ti...,1,0.354008,0.658730,0.114,0.688,0.198,0.9519
3,Basically there's a family where a little boy ...,0,-0.057813,0.454167,0.125,0.816,0.059,-0.9213
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,0.217952,0.452916,0.050,0.806,0.144,0.9744
...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,1,0.394425,0.522897,0.045,0.765,0.189,0.9886
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,-0.276190,0.642857,0.160,0.730,0.110,-0.6837
49997,I am a Catholic taught in parochial elementary...,0,0.056984,0.395538,0.181,0.704,0.115,-0.9734
49998,I'm going to have to disagree with the previou...,0,-0.048663,0.447034,0.116,0.804,0.080,-0.8657


In [5]:
'''
    Begin Neural Net
'''

classes = training["sentiment"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(4, 1)),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])

model.save("IMDBNN.keras")

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=1)
model.fit(x_train, y_train, epochs=100, verbose=2)


c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1329/1329 - 3s - 2ms/step - accuracy: 0.6854 - loss: 0.6463
Epoch 2/100
1329/1329 - 1s - 964us/step - accuracy: 0.7664 - loss: 0.5229
Epoch 3/100
1329/1329 - 1s - 983us/step - accuracy: 0.7700 - loss: 0.4901
Epoch 4/100
1329/1329 - 1s - 992us/step - accuracy: 0.7704 - loss: 0.4849
Epoch 5/100
1329/1329 - 1s - 956us/step - accuracy: 0.7698 - loss: 0.4834
Epoch 6/100
1329/1329 - 1s - 1ms/step - accuracy: 0.7698 - loss: 0.4824
Epoch 7/100
1329/1329 - 1s - 956us/step - accuracy: 0.7710 - loss: 0.4819
Epoch 8/100
1329/1329 - 1s - 989us/step - accuracy: 0.7709 - loss: 0.4818
Epoch 9/100
1329/1329 - 1s - 961us/step - accuracy: 0.7713 - loss: 0.4815
Epoch 10/100
1329/1329 - 1s - 1ms/step - accuracy: 0.7715 - loss: 0.4813
Epoch 11/100
1329/1329 - 3s - 2ms/step - accuracy: 0.7702 - loss: 0.4814
Epoch 12/100
1329/1329 - 1s - 964us/step - accuracy: 0.7705 - loss: 0.4814
Epoch 13/100
1329/1329 - 1s - 989us/step - accuracy: 0.7712 - loss: 0.4813
Epoch 14/100
1329/1329 - 2s - 1ms/step - a

In [6]:

model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

1329/1329 - 1s - 1ms/step - accuracy: 0.7720 - loss: 0.4776
235/235 - 0s - 1ms/step - accuracy: 0.7816 - loss: 0.4716
Train / Test Accuracy: 77.2% / 78.2%


In [ ]:

from sklearn.metrics import confusion_matrix
matrices = []
for i in range(3):
  model = tf.keras.models.load_model("SANN.keras")
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
  model.fit(x_train, y_train, epochs=10)

  y_pred = model.predict(x_test)
  class_preds = np.argmax(y_pred, axis=-1)
  matrices.append(confusion_matrix(y_test, class_preds))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix


